In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.sql('''select 'spark' as hello''')
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



### Pivot dataframe

In [ ]:
_p = (df
      .withColumn('value', F.col('value').cast(T.DoubleType()))
      .withColumn('feature', F.translate('feature', '.', '_'))
      .groupBy('cust_id','dt')
      .pivot('feature')
      .agg(F.expr("first(value)"))
     )
_p.printSchema()

In [ ]:
df3 = (df2
      .select(['dt',
               'cust_id',
               'product',
               'jrnl_id',
               'jrnl_cr_datetime_ux',
               'lag_jrnl_cr_datetime_ux'
              ])
      .withColumn('session_start_flag', 
                   F.when(df2.lag_jrnl_cr_datetime_ux.isNull(), 1)
                   .when(df2.jrnl_cr_datetime_ux-df2.lag_jrnl_cr_datetime_ux>900, 1)
                   .otherwise(0))
      .withColumn('session_no', 
                  F.sum(F.col('session_start_flag'))
                    .over(Window.partitionBy('cust_id','dt')
                                .orderBy('jrnl_cr_datetime_ux','jrnl_id')
                                .rowsBetween(Window.unboundedPreceding, Window.currentRow))
                                )
      .cache()
      )


